# Connection of `puodbc` to an sql server container

Excersice that should work for different MS SQL servers  

## References

* On the connection

https://learn.microsoft.com/en-us/sql/connect/python/pyodbc/step-3-proof-of-concept-connecting-to-sql-using-pyodbc?view=sql-server-ver16


* Drivers installation



## Prerequisites

* The sensitive info. should be stored in a `.env` file. See `_.env_syntax.txt` for details.
* A docker container with MS SQL server should be running. See *create_container.md*. 

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Inputs
load_dotenv()

# Connection
server = 'localhost,1433' # According to docker ps -a
database = 'TestDB'
sql_pass = os.getenv('MY_SQL_PASS')
sql_user = os.getenv('MY_SQL_USER')
payload = [
    'DRIVER={ODBC Driver 18 for SQL Server}',
    'SERVER='+server,
    'DATABASE='+database,
    'UID='+sql_user,
    'PWD='+sql_pass,
    'ENCRYPT=yes',
    'TrustServerCertificate=yes'
    ]

## Reading as pd.Dataframe using sqlalquemy

Because pandas reads without warning using sqlalchemy

In [3]:
import sqlalchemy as sa

In [4]:
connection_url = sa.engine.URL.create("mssql+pyodbc", query={"odbc_connect": ';'.join(payload)})
engine = sa.create_engine(connection_url)

In [5]:

with engine.begin() as conn:
    df = pd.read_sql_query(sa.text("SELECT * FROM Inventory"), conn)
df


,id,name,quantity
0,1,banana,150
1,2,orange,154
2,1,apple,160
3,2,melon,200
4,5,sandias,155
5,5,sandias,155
6,5,sandias,155


## Fetching information, inserting rows and reading in Pandas using pyodbc

In [6]:
import pyodbc

In [7]:
conn = pyodbc.connect(';'.join(payload))
cursor = conn.cursor()

In [8]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2022 (RTM-CU4) (KB5026717) - 16.0.4035.4 (X64) 
	May  1 2023 10:34:45 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 20.04.6 LTS) <X64>


In [9]:
# Inserting a row in TestDB table. 
cursor.execute("USE TestDB")
cursor.execute("INSERT INTO Inventory VALUES (5, 'sandias', 155);")
conn.commit()

In [10]:
# Reading the dataframe
pd.read_sql("SELECT * FROM Inventory",conn)
# UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2

/home/dxota/miniconda3/envs/gnl/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,name,quantity
0,1,banana,150
1,2,orange,154
2,1,apple,160
3,2,melon,200
4,5,sandias,155
5,5,sandias,155
6,5,sandias,155
7,5,sandias,155


In [11]:
# Close the connection
cursor.close()
conn.close()